In [1]:
import librosa
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
def audio_norm(data):
    max_data = np.max(np.absolute(data))
    return data/(max_data+1e-6)*0.5

In [3]:
class Config(object):
    def __init__(self,
                 sampling_rate=16000, audio_duration=4, n_classes=41,
                 use_mfcc=False, n_folds=10, learning_rate=0.0001, 
                 max_epochs=50, n_mfcc=40, datagen_num = 2):
        self.sampling_rate = sampling_rate
        self.audio_duration = audio_duration
        self.n_classes = n_classes
        self.use_mfcc = use_mfcc
        self.n_mfcc = n_mfcc
        self.n_folds = n_folds
        self.learning_rate = learning_rate
        self.max_epochs = max_epochs
        self.datagen_num = datagen_num
        self.audio_length = self.sampling_rate * self.audio_duration
        if self.use_mfcc:
            self.dim = (self.n_mfcc, 1 + int(np.floor(self.audio_length/512)), 1)
        else:
            self.dim = (self.audio_length, 1)

In [27]:
def prepare_data(df, config, data_dir):
    X = np.empty(shape=(df.shape[0] * config.datagen_num, config.dim[0], config.dim[1], 1))
    y = np.empty(df.shape[0] * config.datagen_num)
    input_length = config.audio_length
    for i, fname in enumerate(df.index):
        print(fname+' ({0}/{1})'.format(i,df.shape[0]))
        file_path = data_dir + fname
        data, _ = librosa.core.load(file_path, sr=config.sampling_rate)
        data = audio_norm(data)
        for j in range(config.datagen_num):
            shifted_data = data
            #pitch shift
            #bpo = 24 #how many steps per octave
            #pr = 3/24 #pitch shift range
            #ps = int(np.random.uniform(-pr * bpo, pr * bpo) + 0.5) #how many (fractional) half-steps to shift y
            #shifted_data = librosa.effects.pitch_shift(shifted_data, config.sampling_rate, n_steps = ps, bins_per_octave = bpo)
            # time stretch
            tr = 1.1 #speed up/down rate
            lgtr = np.log(tr)
            ts = 2 ** np.random.uniform(-lgtr,lgtr)
            shifted_data = librosa.effects.time_stretch(shifted_data, ts)
            #white noise
            #wnvr = 0.05 # white noise volume range
            #wnv  = np.random.uniform(0, wnvr) # white noise volume, random
            #shifted_data += np.random.uniform(-wnv, wnv, shifted_data.shape)
            # Random offset / Padding
            if len(shifted_data) < input_length:
                ratio = input_length/len(shifted_data)
                ratio = np.ceil(ratio)
                shifted_data = np.tile(shifted_data,int(ratio))
            max_offset = len(shifted_data) - input_length
            offset = np.random.randint(max_offset)
            shifted_data = shifted_data[offset:(input_length+offset)]
            #mfcc
            shifted_data = librosa.feature.mfcc(shifted_data, sr=config.sampling_rate, n_mfcc=config.n_mfcc)
            shifted_data = np.expand_dims(shifted_data, axis=-1)
            X[ i * config.datagen_num + j, :] = shifted_data
            y[ i * config.datagen_num + j] = df.label_idx[i]
    return X, y

In [28]:
def prepare_test_data(config, data_dir='../input/audio_test/'):
    df = pd.read_csv('../input/test.csv')
    test_data = np.empty(shape=(df.shape[0], config.dim[0], config.dim[1], 1))
    input_length = config.audio_length
    for i, fname in enumerate(df['fname']):
        print(fname)
        file_path = data_dir + fname
        data, _ = librosa.core.load(file_path, sr=config.sampling_rate)
        if len(data)==0:
            data = np.zeros(88200)
        data = audio_norm(data)
        # Random offset / Padding
        if len(data) < input_length:
            ratio = input_length/len(data)
            ratio = np.ceil(ratio)
            data = np.tile(data,int(ratio))
        max_offset = len(data) - input_length
        offset = np.random.randint(max_offset)
            data = data[offset:(input_length+offset)]
        data = librosa.feature.mfcc(data, sr=config.sampling_rate, n_mfcc=config.n_mfcc)
        data = np.expand_dims(data, axis=-1)
        test_data[i] = data
    return test_data

In [29]:
config = Config(sampling_rate=44100, audio_duration=4, n_folds=5, 
                learning_rate=0.001, use_mfcc=True, n_mfcc=40, datagen_num=5)
train = pd.read_csv("../input/train.csv")
LABELS = list(train.label.unique())
label_idx = {label: i for i, label in enumerate(LABELS)}
train.set_index("fname", inplace=True)
train["label_idx"] = train.label.apply(lambda x: label_idx[x])

In [33]:
test_data = prepare_test_data(config, data_dir='../input/audio_test/')

00063640.wav
0013a1db.wav
002bb878.wav
002d392d.wav
00326aa9.wav
0038a046.wav
003995fa.wav
005ae625.wav
007759c4.wav
008afd93.wav
00a161c0.wav
00a7a2f6.wav
00ae03f6.wav
00b2404e.wav
00beb030.wav
00c4d5b8.wav
00c92c05.wav
00ccf065.wav
00d0ab77.wav
00dffe3a.wav
00e33205.wav
00e7ed07.wav
00eac343.wav
00f2f692.wav
0102a895.wav
010a0b3a.wav
010eed01.wav
01115622.wav
01198b7e.wav
01207ce5.wav
0137bfba.wav
014eb52e.wav
01608fa3.wav
0169348e.wav
016e52da.wav
0183b482.wav
0185de6b.wav
01a5a2a3.wav
01aed32a.wav
01afccc5.wav
01b4cdb5.wav
01b85217.wav
01bb344f.wav
01bede12.wav
01c9d6d3.wav
01e628cb.wav
01e6e112.wav
01e99e2d.wav
01eccb6e.wav
01f27f99.wav
01f29edb.wav
01f9883e.wav
01fd22ba.wav
01fe4a01.wav
02107093.wav
02182ebd.wav
02198549.wav
0220d48a.wav
02210ee7.wav
022170df.wav
023431b2.wav
023e7db3.wav
023eab1f.wav
02482fdc.wav
024e45b1.wav
026820e6.wav
02697329.wav
027a8b14.wav
027dbe3e.wav
027f06cc.wav
028278f0.wav
028db587.wav
02960f07.wav
0298b2c6.wav
02a0eb3c.wav
02ad476f.wav
02af55f6.wav

1146e063.wav
11544687.wav
115a258b.wav
115aa41f.wav
116b77d5.wav
11704d14.wav
11770392.wav
117dcac8.wav
11844b37.wav
11867b61.wav
1189cf83.wav
118f1763.wav
11986117.wav
119a3278.wav
11a10ce8.wav
11a4f814.wav
11ae787e.wav
11cc6d11.wav
11e6a1ec.wav
11e80486.wav
11e95378.wav
11edc852.wav
11f661bd.wav
11f6b674.wav
11fa11a0.wav
12034b1a.wav
120c7292.wav
1221a798.wav
122abe8c.wav
122f1508.wav
12394644.wav
123986b6.wav
1242ac46.wav
1248a779.wav
1248f9e2.wav
12508a7b.wav
12584f90.wav
125c89cb.wav
125dccd9.wav
12679aee.wav
126a48d2.wav
1271e7aa.wav
12770ffc.wav
12798b6a.wav
127a43ec.wav
127d3803.wav
12a0327a.wav
12a83756.wav
12ac9a66.wav
12adec75.wav
12ae3ff9.wav
12bb4b63.wav
12bf9735.wav
12bfa703.wav
12c73c11.wav
12d71e7f.wav
12d94273.wav
12e91a56.wav
12ed3971.wav
12edfe79.wav
12f5ec05.wav
12f71a9d.wav
12f9efda.wav
12fb2e35.wav
130112d1.wav
1302eade.wav
130988c6.wav
13184046.wav
13331371.wav
13354f83.wav
133599c8.wav
13371a0b.wav
133a1dac.wav
13489639.wav
138bb73e.wav
1391b6b2.wav
13979584.wav

225e3135.wav
225f0626.wav
2261073e.wav
226ea2c6.wav
227f2c50.wav
228153f2.wav
22824c18.wav
22891802.wav
2293e34d.wav
22abde0a.wav
22b2efa2.wav
22b5a548.wav
22c3952c.wav
22c798ac.wav
22ca2dbb.wav
22e88b33.wav
22f3b36c.wav
22f70afe.wav
23029663.wav
23071dd2.wav
23094c55.wav
230ce5d5.wav
23176e01.wav
231d230c.wav
231e9e57.wav
231f314d.wav
232750a8.wav
232ec97b.wav
23300ded.wav
233996cf.wav
233a83b7.wav
233d0591.wav
233d8cbb.wav
235dbbf0.wav
2369256d.wav
23732830.wav
23824542.wav
23852f22.wav
238d7b37.wav
238d8cad.wav
238deaad.wav
23984d44.wav
239988bc.wav
2399acef.wav
23bae3b1.wav
23bf31da.wav
23ca551e.wav
23cdfc67.wav
23d1d920.wav
23d4cbff.wav
23e94e4f.wav
23ede927.wav
23f2b1f7.wav
23f592dd.wav
23f8b4b0.wav
23fc49d9.wav
240c45cb.wav
240d2bb2.wav
240dfb34.wav
241224d6.wav
242118f2.wav
2421d14b.wav
2423598a.wav
2429265b.wav
242ea92c.wav
2436090e.wav
243ab0b7.wav
2442e7c6.wav
244618d7.wav
244baebc.wav
244dc716.wav
24599c96.wav
24602db6.wav
2472cdfe.wav
247431d8.wav
247a90a0.wav
2487cd7b.wav

34be5213.wav
34c7a37b.wav
34cddac7.wav
34d1fffa.wav
34d84768.wav
34f6f431.wav
34ff2f37.wav
350b89b8.wav
350ed42d.wav
35115db0.wav
351b0ba1.wav
351dcc25.wav
35235d3f.wav
352c769d.wav
352f143a.wav
3531e0f4.wav
3533dc16.wav
3538f800.wav
353b44b7.wav
353f55db.wav
3542eddc.wav
35497fc4.wav
355eeedf.wav
355f0738.wav
356cab15.wav
3579ff4b.wav
3592e0cb.wav
3597c4e5.wav
3598cc58.wav
359c9ba3.wav
35b91bb1.wav
35bb05b0.wav
35bf4ae5.wav
35c0f6b7.wav
35cb8bd1.wav
35ce58ae.wav
35d008df.wav
35d0858d.wav
35da43f9.wav
35def6dc.wav
35e1140b.wav
35e4b0fd.wav
35f3a48c.wav
35f6a4b8.wav
35ff3f29.wav
35ffae5a.wav
360348af.wav
360e9c8e.wav
360ed752.wav
361285ac.wav
3617db48.wav
361c028f.wav
361fbd17.wav
3620c12b.wav
3623b02e.wav
363a3760.wav
364d2e90.wav
36507474.wav
365a4c89.wav
365f9044.wav
3660d731.wav
3668b282.wav
36690c27.wav
3670cc04.wav
367ba81f.wav
36a0c334.wav
36a2202d.wav
36a92b29.wav
36a98063.wav
36b4867e.wav
36ba1f72.wav
36cd9e16.wav
36d47091.wav
36dbdb68.wav
36e470e4.wav
36e7790c.wav
36e79aec.wav

462a705d.wav
462c7777.wav
462ec068.wav
46321665.wav
46380229.wav
463d4298.wav
463de778.wav
463e231d.wav
4646a6e3.wav
464ae66b.wav
465c6192.wav
46622a4c.wav
466647df.wav
4666bee5.wav
466e8bab.wav
467c5a5a.wav
4683f13c.wav
4698193e.wav
4699c352.wav
46a20346.wav
46a24375.wav
46a4d46b.wav
46ab7601.wav
46aebc33.wav
46b06055.wav
46b5a369.wav
46d15a22.wav
46e1f70a.wav
46f07933.wav
46f0b51b.wav
4706cebd.wav
471a6fca.wav
471ab81d.wav
47268a0a.wav
47333aac.wav
4733f511.wav
473bd952.wav
473e9a47.wav
4741ae62.wav
47494a3a.wav
47538a81.wav
47542ddc.wav
47744c05.wav
47793988.wav
47795659.wav
47913bc5.wav
4797fc8b.wav
479ab3cf.wav
479d9994.wav
47a26734.wav
47ac630d.wav
47acbfcd.wav
47af6236.wav
47b7eb95.wav
47bba234.wav
47c48965.wav
47c57028.wav
47ca6f5b.wav
47e5c682.wav
47eba90e.wav
47ebd5f5.wav
47f5e3e1.wav
47f7b3fa.wav
47fa98ea.wav
480fb007.wav
481623f9.wav
48165dd3.wav
481c84da.wav
481f3844.wav
48395d1c.wav
483c028d.wav
483feb77.wav
48432e76.wav
484e69ce.wav
484e96f4.wav
4854915c.wav
486c058b.wav

...

ac4e170d.wav
ac507f36.wav
ac507fd0.wav
ac590977.wav
ac71fa9c.wav
ac755dd6.wav
ac7b9e18.wav
ac7c8ae3.wav
ac7eec59.wav
ac822579.wav
ac8b9268.wav
ac8c9f4c.wav
ac8cce64.wav
ac909b62.wav
ac9151cf.wav
ac916661.wav
ac9f8827.wav
aca050ea.wav
aca46609.wav
acb2260c.wav
acb2ae9a.wav
acb33e73.wav
acb60d34.wav
acba91ee.wav
accb746f.wav
acd9adb9.wav
acdcfb8f.wav
ace88278.wav
acf4569a.wav
acf4890c.wav
ad0ab6fa.wav
ad142ed0.wav
ad1a8bce.wav
ad1f2187.wav
ad23f99c.wav
ad2ee4b2.wav
ad323692.wav
ad4c828c.wav
ad4d97de.wav
ad5821b3.wav
ad617575.wav
ad6e3b22.wav
ad71a0ba.wav
ad72561f.wav
ad7b2dfe.wav
ad7bd363.wav
ad7dbf27.wav
ad814adf.wav
ad8a6cdd.wav
ad8b78e2.wav
ad8d702a.wav
ad8de222.wav
ad936e6a.wav
ada15329.wav
ada2b04f.wav
adbf84f5.wav
adc41a55.wav
add0a091.wav
add0efb3.wav
add3bd1d.wav
add600ff.wav
ade59799.wav
adf7b2fa.wav
ae17caec.wav
ae1e828a.wav
ae23366f.wav
ae25e2b8.wav
ae2e8d4f.wav
ae302df0.wav
ae331164.wav
ae374569.wav
ae3dc480.wav
ae43398f.wav
ae48e08f.wav
ae4e8c4f.wav
ae4fbd56.wav
ae57c6d1.wav

bcfe81f2.wav
bd062294.wav
bd0a9017.wav
bd0fa421.wav
bd16d29a.wav
bd1d9f10.wav
bd23c357.wav
bd271732.wav
bd3ba846.wav
bd489bec.wav
bd575d0a.wav
bd58d191.wav
bd5e02e4.wav
bd63ed5f.wav
bd661885.wav
bd6b558b.wav
bd6c1270.wav
bd702317.wav
bd70a3ae.wav
bd772d30.wav
bd789949.wav
bd79cbb8.wav
bd8a0d61.wav
bd9cf441.wav
bdb3d952.wav
bdbb5a2a.wav
bdc28adc.wav
bdc2a6f2.wav
bdc3a110.wav
bdc5b4a7.wav
bdd0b24e.wav
bdd2a664.wav
bdd2f431.wav
bdd4e518.wav
bdee019c.wav
bdf92503.wav
be09bb9d.wav
be0c6f8d.wav
be12134a.wav
be176425.wav
be264bf2.wav
be2ea892.wav
be437c69.wav
be46aa9e.wav
be4c1d3d.wav
be4f04a8.wav
be53a71f.wav
be571523.wav
be640229.wav
be666047.wav
be66ee27.wav
be7277bc.wav
be771343.wav
be83e54e.wav
be8b491b.wav
be8c3977.wav
be8f6c7c.wav
be8fcd90.wav
be92929b.wav
be9482c7.wav
be95f53a.wav
be977e89.wav
be97f271.wav
be9d0575.wav
bea3dcfa.wav
bea9dbc3.wav
bebf2559.wav
bec4f3ab.wav
bec6e952.wav
bed3738c.wav
bedf541e.wav
bee10a06.wav
bee40633.wav
bee570da.wav
bee58101.wav
bee64b40.wav
bee6c09a.wav

ce4b2f15.wav
ce5b2140.wav
ce6859a8.wav
ce8172d5.wav
ce86e76a.wav
ce890eb4.wav
ce95dcea.wav
ce99f94c.wav
ce9c1ce8.wav
ce9dd7b8.wav
cea165f8.wav
cea67a35.wav
ceab0387.wav
ceb6e9c1.wav
cec9647b.wav
cecb67c0.wav
cecd4f9f.wav
cee05a5f.wav
cee063a7.wav
ceec151c.wav
ceef96eb.wav
cefb71c5.wav
ceffb395.wav
cf0471b4.wav
cf0d3082.wav
cf0f872c.wav
cf109573.wav
cf156f7d.wav
cf225634.wav
cf2c1168.wav
cf370066.wav
cf3933c7.wav
cf3f6594.wav
cf4eb36a.wav
cf510fc8.wav
cf583cf8.wav
cf5ec323.wav
cf654f37.wav
cf655c72.wav
cf6a21d7.wav
cf6c9166.wav
cf6e8727.wav
cf7b4e97.wav
cf830a16.wav
cf8357ba.wav
cf83e7ed.wav
cf99e424.wav
cf9fa471.wav
cfada6b4.wav
cfadac1f.wav
cfb02ecb.wav
cfb76128.wav
cfc16683.wav
cfc20796.wav
cfc2388d.wav
cfd05410.wav
cfd7f8d1.wav
cfe26f15.wav
cfe27a89.wav
cfe540af.wav
cfeb4cda.wav
cfee75b9.wav
cfef6dc7.wav
d001e455.wav
d002c5a8.wav
d008405a.wav
d012cd6f.wav
d012d82d.wav
d014078b.wav
d01571db.wav
d0296d9a.wav
d02c2598.wav
d0331f5c.wav
d033f19a.wav
d034f755.wav
d037cc6b.wav
d0420016.wav

e0273514.wav
e0278ad9.wav
e02910e8.wav
e035f97b.wav
e03c8bf3.wav
e0419e15.wav
e043afb3.wav
e0457b4a.wav
e059d9a7.wav
e06b1539.wav
e06b693c.wav
e06bfc3d.wav
e06deab0.wav
e0802994.wav
e083ae15.wav
e08808ae.wav
e08fced0.wav
e0a72225.wav
e0ad96e1.wav
e0bfc4ec.wav
e0d34abd.wav
e0d8ea27.wav
e0d8ff62.wav
e0d96e55.wav
e0de96a6.wav
e0e373a5.wav
e0e6a1de.wav
e0eb0c8a.wav
e0ee247e.wav
e0fc67d9.wav
e103173b.wav
e10f3d3d.wav
e112e5a2.wav
e1145387.wav
e11bf16f.wav
e11ccb9d.wav
e1211dc4.wav
e122ef4b.wav
e128a888.wav
e13193af.wav
e1423cf1.wav
e14ebc15.wav
e15de084.wav
e15fedd2.wav
e1611806.wav
e173748a.wav
e19618a0.wav
e1ab90cd.wav
e1b1ee02.wav
e1c172fa.wav
e1c78f41.wav
e1ca89e3.wav
e1d88aa8.wav
e1eccf73.wav
e1f3b477.wav
e1fc13fd.wav
e218f5ab.wav
e21c7c53.wav
e22b1cc6.wav
e233206c.wav
e23393b9.wav
e245a3f5.wav
e2529ad3.wav
e25b3e5d.wav
e25f0efb.wav
e26962f7.wav
e269a6c9.wav
e269cdb6.wav
e26b56a0.wav
e275b9dd.wav
e27a0b20.wav
e291574a.wav
e29e1f8b.wav
e2adb002.wav
e2b92cb5.wav
e2b9b01a.wav
e2c0033d.wav

f26f9711.wav
f27a6fcd.wav
f27c1378.wav
f2849aa9.wav
f290369d.wav
f29686d0.wav
f29881fc.wav
f2a1e334.wav
f2ae2b94.wav
f2b378fb.wav
f2b6fd89.wav
f2ba99d8.wav
f2be783d.wav
f2bf7e84.wav
f2c60da6.wav
f2c781bc.wav
f2ce1323.wav
f2e678fe.wav
f2f5a41a.wav
f305a054.wav
f30f5fca.wav
f32309a8.wav
f324562c.wav
f330bf67.wav
f3368ed3.wav
f33acfe4.wav
f33fadfc.wav
f33fca42.wav
f342c01b.wav
f3482344.wav
f34b6f3d.wav
f34fb400.wav
f3512104.wav
f351c242.wav
f35775ce.wav
f35f18f6.wav
f363b68d.wav
f36cfcb5.wav
f36f1fc6.wav
f37780ed.wav
f3799674.wav
f37abe8d.wav
f37cc0fa.wav
f38c11c3.wav
f395c655.wav
f3967f91.wav
f3a871f0.wav
f3ab1ef3.wav
f3ac0e7d.wav
f3b36b54.wav
f3b615ee.wav
f3b7a589.wav
f3b85215.wav
f3bc7650.wav
f3c20960.wav
f3c8bebe.wav
f3ccf80f.wav
f3dc4e70.wav
f3e88606.wav
f3fb49b4.wav
f40ccfa4.wav
f40cdd0e.wav
f40d9111.wav
f40f792d.wav
f41ae5f1.wav
f4235f93.wav
f4272031.wav
f42b9f0e.wav
f4344d88.wav
f43c73b6.wav
f43f193a.wav
f442491d.wav
f4467979.wav
f446a833.wav
f44f2fc4.wav
f45176ff.wav
f471cb4d.wav

In [ ]:
np.save('../data_gen/test_mfcc.npy',test_data)
del test_data

In [ ]:
X_train, y = prepare_data(train, config, '../input/audio_train/')

In [ ]:
np.save('../data_gen/train_mfcc_stretch=1.1_num=5_len=4.npy',X_train)
np.save('../data_gen/label_stretch=1.1_num=5_len=4.npy',y)